In [4]:

# МИНИМАЛЬНЫЙ ПАРСЕР НЕСКОЛЬКИХ HTML ФАЙЛОВ

# 1. Импорт библиотек
import pandas as pd
from bs4 import BeautifulSoup
import os
import re

#### 2.1 Формирование структуры данных
Ознакомьтесь с возможными для получения историческими данными о погоде с ресурса Open-Meteo (https://open-meteo.com/en/docs/historical-weather-api). Необходимо проанализировать доступные для получения почасовые данные и составить структуру данных, привести описание атрибутов в структуре данных, обосновать их выбор, соотнести с получаемыми историческими данными по аэропортам в Модуле 1.

#### Минимальная структура данных Open-Meteo для авиации

#### 1. Базовые поля (обязательные)

| Поле | Тип | Описание | Соответствие Модуль 1 |
|------|-----|----------|----------------------|
| `airport_code` | string | Код аэропорта (ICAO/IATA) | `airport` |
| `timestamp` | datetime | Время наблюдения (UTC) | `date` + `time` |
| `latitude` | float | Широта | - |
| `longitude` | float | Долгота | - |

#### 2. Основные метеопараметры

##### Температура и влажность:
| Поле | Тип | Единицы | Соответствие Модуль 1 |
|------|-----|---------|----------------------|
| `temperature_2m` | float | °C | `temperature_c` |
| `relative_humidity_2m` | float | % | `humidity` |
| `dew_point_2m` | float | °C | `dew_point` |

##### Ветер (критично для авиации):
| Поле | Тип | Единицы | Соответствие Модуль 1 |
|------|-----|---------|----------------------|
| `wind_speed_10m` | float | м/с | `wind_speed_ms` |
| `wind_direction_10m` | float | ° | `wind_direction` |
| `wind_gusts_10m` | float | м/с | `wind_gust` |

##### Давление:
| Поле | Тип | Единицы | Соответствие Модуль 1 |
|------|-----|---------|----------------------|
| `surface_pressure` | float | гПа | `pressure_station` |
| `pressure_msl` | float | гПа | `pressure_sea` |

##### Видимость и облачность:
| Поле | Тип | Единицы | Соответствие Модуль 1 |
|------|-----|---------|----------------------|
| `visibility` | float | м | `visibility_km` |
| `cloud_cover` | float | % | `cloud_cover` |

##### Осадки:
| Поле | Тип | Единицы | Соответствие Модуль 1 |
|------|-----|---------|----------------------|
| `precipitation` | float | мм | - |
| `weather_code` | integer | - | `weather_phenomena` |

#### 3. Обоснование минимального набора

##### Критические для авиации параметры:

1. **Ветер** (3 параметра):
   - Скорость - ограничения для взлета/посадки
   - Направление - выбор ВПП
   - Порывы - безопасность

2. **Видимость** (1 параметр):
   - Минимальные требования для полетов
   - CAT I/II/III подходы

3. **Облачность** (1 параметр):
   - Высота нижней границы
   - Инструментальные/визуальные полеты

4. **Температура/влажность** (3 параметра):
   - Обледенение
   - Плотность воздуха
   - Производительность ВС

5. **Давление** (2 параметра):
   - Высотомеры
   - QNH/QFE настройки

#### 4. Соответствие с Модулем 1

##### Прямые соответствия (9 параметров):

| Модуль 1 | Open-Meteo | Конвертация |
|----------|------------|-------------|
| `temperature_c` | `temperature_2m` | Прямое |
| `humidity` | `relative_humidity_2m` | Прямое |
| `pressure_station` | `surface_pressure` | 1 гПа = 0.75 мм рт.ст. |
| `pressure_sea` | `pressure_msl` | 1 гПа = 0.75 мм рт.ст. |
| `wind_speed_ms` | `wind_speed_10m` | Прямое |
| `wind_direction` | `wind_direction_10m` | Прямое |
| `visibility_km` | `visibility` | 1 км = 1000 м |
| `dew_point` | `dew_point_2m` | Прямое |
| `cloud_cover` | `cloud_cover` | Прямое |

##### Дополнительные данные Open-Meteo:

1. **Координаты** - точное местоположение
2. **Порывы ветра** - важны для безопасности
3. **Коды погоды** - стандартизированная классификация
4. **Осадки в мм** - количественная оценка



#### 2.2 Сбор данных, используя API

Используя API Open-Meteo, получите данные о почасовых показаниях погоды для 8 городов, анализируемых в Модуле 1, за период с 16.01.2024 по 19.01.2024. Данные соберите в один набор данных. Сформируйте набор данных о погодных условиях в соответствии с разработанной структурой по всем предоставленным данным.

In [7]:
"""
СБОР ДАННЫХ OPEN-METEO
Для 8 российских аэропортов (16-19 января 2024)
"""

# 1. Импорт библиотек
import pandas as pd
import requests
import time

# 2. Координаты аэропортов
airports = {
    "Архангельск": {"lat": 64.6000, "lon": 40.7167},
    "Казань": {"lat": 55.6061, "lon": 49.2787},
    "Магадан": {"lat": 59.9100, "lon": 150.7200},
    "Нижний_Новгород": {"lat": 56.2300, "lon": 43.7800},
    "Пулково": {"lat": 59.8000, "lon": 30.2650},
    "Сочи": {"lat": 43.4500, "lon": 39.9565},
    "Толмачёво": {"lat": 55.0125, "lon": 82.6500},
    "Шереметьево": {"lat": 55.9725, "lon": 37.4131}
}

# 3. Параметры API
BASE_URL = "https://archive-api.open-meteo.com/v1/archive"
params_template = {
    "start_date": "2024-01-16",
    "end_date": "2024-01-19",
    "hourly": "temperature_2m,relative_humidity_2m,wind_speed_10m,wind_direction_10m,visibility,cloud_cover",
    "timezone": "Europe/Moscow",
    "wind_speed_unit": "ms"
}

In [8]:
# 4. Функция сбора данных
def collect_weather_data():
    """Собирает данные для всех аэропортов."""
    
    all_data = []
    
    for name, coords in airports.items():
        print(f"Сбор данных для: {name}")
        
        # Формируем параметры запроса
        params = params_template.copy()
        params.update({
            "latitude": coords["lat"],
            "longitude": coords["lon"]
        })
        
        try:
            # Запрос к API
            response = requests.get(BASE_URL, params=params, timeout=10)
            
            if response.status_code == 200:
                data = response.json()
                
                # Создаем DataFrame
                df = pd.DataFrame(data["hourly"])
                df["airport"] = name
                df["latitude"] = coords["lat"]
                df["longitude"] = coords["lon"]
                
                all_data.append(df)
                print(f"  Успешно: {len(df)} записей")
                
            else:
                print(f"  Ошибка: {response.status_code}")
                
        except Exception as e:
            print(f"  Ошибка: {e}")
        
        # Пауза между запросами
        time.sleep(0.5)
    
    # Объединение данных
    if all_data:
        combined = pd.concat(all_data, ignore_index=True)
        print(f"\nВсего собрано записей: {len(combined)}")
        return combined
    else:
        print("Не удалось собрать данные")
        return pd.DataFrame()

In [9]:
# 5. Сбор данных
print("Начало сбора данных...")
weather_df = collect_weather_data()

Начало сбора данных...
Сбор данных для: Архангельск
  Успешно: 96 записей
Сбор данных для: Казань
  Успешно: 96 записей
Сбор данных для: Магадан
  Успешно: 96 записей
Сбор данных для: Нижний_Новгород
  Успешно: 96 записей
Сбор данных для: Пулково
  Успешно: 96 записей
Сбор данных для: Сочи
  Успешно: 96 записей
Сбор данных для: Толмачёво
  Успешно: 96 записей
Сбор данных для: Шереметьево
  Успешно: 96 записей

Всего собрано записей: 768


In [10]:
weather_df.head()

,time,temperature_2m,relative_humidity_2m,wind_speed_10m,wind_direction_10m,visibility,cloud_cover,airport,latitude,longitude
0,2024-01-16T00:00,-17.3,83,3.52,97,None,100,Архангельск,64.6,40.7167
1,2024-01-16T01:00,-17.3,83,3.61,95,None,100,Архангельск,64.6,40.7167
2,2024-01-16T02:00,-16.9,82,3.60,92,None,100,Архангельск,64.6,40.7167
3,2024-01-16T03:00,-16.5,82,3.80,90,None,100,Архангельск,64.6,40.7167
4,2024-01-16T04:00,-17.0,83,3.80,92,None,100,Архангельск,64.6,40.7167


In [11]:
# 6. Обработка и сохранение
if not weather_df.empty:
    # Переименование колонок
    weather_df = weather_df.rename(columns={
        "time": "timestamp",
        "temperature_2m": "temperature",
        "relative_humidity_2m": "humidity",
        "wind_speed_10m": "wind_speed_ms",
        "wind_direction_10m": "wind_direction",
        "visibility": "visibility_m",
        "cloud_cover": "cloud_cover_percent"
    })
    
    # Конвертация времени
    weather_df["timestamp"] = pd.to_datetime(weather_df["timestamp"])
    
    # Добавление вычисляемых полей
    weather_df["visibility_km"] = weather_df["visibility_m"] / 1000
    
    # Сохранение
    weather_df.to_csv("russia_airports_weather_2024_01.csv", index=False, encoding='utf-8')
    print(f"\nДанные сохранены в: russia_airports_weather_2024_01.csv")
    
    # Базовая информация
    print(f"\n=== ИНФОРМАЦИЯ О ДАННЫХ ===")
    print(f"Размер: {weather_df.shape}")
    print(f"Колонки: {weather_df.columns.tolist()}")
    print(f"\nАэропорты: {weather_df['airport'].unique().tolist()}")
    print(f"Период: {weather_df['timestamp'].min()} - {weather_df['timestamp'].max()}")
    
    # Статистика по аэропортам
    print(f"\n=== СТАТИСТИКА ===")
    stats = weather_df.groupby("airport").agg({
        "temperature": ["mean", "min", "max"],
        "wind_speed_ms": ["mean", "max"],
        "visibility_km": ["mean", "min"]
    }).round(1)
    
    print(stats)


Данные сохранены в: russia_airports_weather_2024_01.csv

=== ИНФОРМАЦИЯ О ДАННЫХ ===
Размер: (768, 11)
Колонки: ['timestamp', 'temperature', 'humidity', 'wind_speed_ms', 'wind_direction', 'visibility_m', 'cloud_cover_percent', 'airport', 'latitude', 'longitude', 'visibility_km']

Аэропорты: ['Архангельск', 'Казань', 'Магадан', 'Нижний_Новгород', 'Пулково', 'Сочи', 'Толмачёво', 'Шереметьево']
Период: 2024-01-16 00:00:00 - 2024-01-19 23:00:00

=== СТАТИСТИКА ===
                temperature             wind_speed_ms       visibility_km     
                       mean   min   max          mean   max          mean  min
airport                                                                       
Архангельск           -11.7 -18.9  -7.2           3.5   6.5           NaN  NaN
Казань                 -8.8 -23.1  -2.2           8.6  14.6           NaN  NaN
Магадан               -20.4 -28.7 -11.9           1.6   4.5           NaN  NaN
Нижний_Новгород        -6.2 -18.0   1.5           5.4   9.3 

#### 2.3 Визуальный анализ многомерных данных
Используя программные средства, визуализируйте зависимости атрибутов в наборе данных. Используйте визуализацию многомерных данных 2 или более способов. Обоснуйте выбранные способы визуализации. Приведите интерпретацию полученным результатам. Сравните данные с данными Модуля 1.

In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Настройка стилей графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

In [ ]:
# Загрузка данных
# Первый файл: данные с API
df_api = pd.read_csv('russia_airports_weather_2024_01.csv')
print(f"Размер первого датасета: {df_api.shape}")
print(f"Колонки первого датасета: {df_api.columns.tolist()}")


In [ ]:

# Второй файл: данные Модуля 1
df_module1 = pd.read_csv('weather_data_min.csv')
print(f"\nРазмер второго датасета: {df_module1.shape}")
print(f"Колонки второго датасета: {df_module1.columns.tolist()}")